In [13]:
import time 
 
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

import re
import os
import io
import PIL.Image as Image

from array import array
import base64

import cv2

In [15]:
#initialize webdriver
# start by defining the options 
options = webdriver.ChromeOptions() 
options.headless = True # it's more scalable to work in headless mode 
# normally, selenium waits for all resources to download 
# we don't need it as the page also populated with the running javascript code. 
options.page_load_strategy = 'none' 
# this returns the path web driver downloaded 
chrome_path = ChromeDriverManager().install() 
chrome_service = Service(chrome_path) 
# pass the defined options and service objects to initialize the web driver 
driver = Chrome(options=options, service=chrome_service) 
driver.implicitly_wait(5)

/tmp/ipykernel_104955/210964506.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True # it's more scalable to work in headless mode


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x559a336b74e3 <unknown>
#1 0x559a333e6c76 <unknown>
#2 0x559a3340d757 <unknown>
#3 0x559a3340c029 <unknown>
#4 0x559a3344accc <unknown>
#5 0x559a3344a47f <unknown>
#6 0x559a33441de3 <unknown>
#7 0x559a334172dd <unknown>
#8 0x559a3341834e <unknown>
#9 0x559a336773e4 <unknown>
#10 0x559a3367b3d7 <unknown>
#11 0x559a33685b20 <unknown>
#12 0x559a3367c023 <unknown>
#13 0x559a3364a1aa <unknown>
#14 0x559a336a06b8 <unknown>
#15 0x559a336a0847 <unknown>
#16 0x559a336b0243 <unknown>
#17 0x7f77b688f18a <unknown>


In [ ]:
url = "https://www.ua.pt/pt/deti/pessoas?staff=7"
d=driver.get(url)
driver.get(url) 
time.sleep(10)

In [ ]:
#get all links in page
lnks=driver.find_elements(By.TAG_NAME, "a")

In [ ]:
def urlmatch(url):
    regex=re.compile(r'(https://www.ua.pt/pt/p/*)'
                     r'([0-9])'
     ,re.IGNORECASE)
    
    return(re.match(regex,url) is not None)


In [ ]:
#get links from people 
peoplelinks=set()
for l in lnks:
    p=l.get_attribute('href')
    if urlmatch(p):
        peoplelinks.add(p)

In [ ]:
len(peoplelinks) #verification

In [ ]:
#gets image from blob link

def get_file_content_chrome(driver, uri):
  result = driver.execute_async_script("""
    var uri = arguments[0];
    var callback = arguments[1];
    var toBase64 = function(buffer){for(var r,n=new Uint8Array(buffer),t=n.length,a=new Uint8Array(4*Math.ceil(t/3)),i=new Uint8Array(64),o=0,c=0;64>c;++c)i[c]="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/".charCodeAt(c);for(c=0;t-t%3>c;c+=3,o+=4)r=n[c]<<16|n[c+1]<<8|n[c+2],a[o]=i[r>>18],a[o+1]=i[r>>12&63],a[o+2]=i[r>>6&63],a[o+3]=i[63&r];return t%3===1?(r=n[t-1],a[o]=i[r>>2],a[o+1]=i[r<<4&63],a[o+2]=61,a[o+3]=61):t%3===2&&(r=(n[t-2]<<8)+n[t-1],a[o]=i[r>>10],a[o+1]=i[r>>4&63],a[o+2]=i[r<<2&63],a[o+3]=61),new TextDecoder("ascii").decode(a)};
    var xhr = new XMLHttpRequest();
    xhr.responseType = 'arraybuffer';
    xhr.onload = function(){ callback(toBase64(xhr.response)) };
    xhr.onerror = function(){ callback(xhr.status) };
    xhr.open('GET', uri);
    xhr.send();
    """, uri)
  if type(result) == int :
    raise Exception("Request failed with status %s" % result)
  return base64.b64decode(result)
#bytes = get_file_content_chrome(driver, blo)


In [ ]:
###saves image with prof name

#get name of person
#name=driver.find_elements(By.TAG_NAME, "h1")
#for n in name:
#    namee=n.text


def readimage(path):
    count = os.stat(path).st_size / 2
    with open(path, "rb") as f:
        return bytearray(f.read())


#image = Image.open(io.BytesIO(bytes))
#image.save(namee+'.png')

In [ ]:
def downloadimage(url,path):
    os.chdir(path)
    driver.get(url) 
    time.sleep(10)
    
    #get blob link
    blo=driver.find_element(By.CSS_SELECTOR, '[alt="image"]').get_attribute("src")

        
    bytes = get_file_content_chrome(driver, blo)
    
    #get name of person
    name=driver.find_elements(By.TAG_NAME, "h1")
    for n in name:
        namee=n.text
    print(namee)
        
    image = Image.open(io.BytesIO(bytes))
    #im=cv2.imread(image)
    #ppath=path+namee+'.png'
    image.save(namee+'.png')
    #cv2.imwrite(path,im)
    print('image saved')
    return

In [ ]:
path='/home/madalena/uni/projeto/deti'
for p in peoplelinks:
    print(p)
    try:
        downloadimage(p,path)
    except NoSuchElementException:
        pass


# data augmentation
https://neptune.ai/blog/data-augmentation-in-python 
https://towardsdatascience.com/top-python-libraries-for-image-augmentation-in-computer-vision-2566bed0533e


## Augly

https://augly.readthedocs.io/en/latest/augly.image.html

In [9]:
import augly.image as imaug
import os
from PIL import Image as im

In [5]:
path="/home/madalena/uni/projeto/deti/output/"
images=[]
for i in os.listdir(path):
    #print(i)
    #image_path=os.path.join(path,i)
    #print(image_path)
    images.append(i)

FileNotFoundError: [Errno 2] No such file or directory: '/home/madalena/uni/projeto/deti/output/'

In [30]:
image=im.open(input_path+"Adão Paulo Soares da Silva.png")
image.show()

In [49]:
input_path = "/home/madalena/Documents/projeto/deti/"
output_path = "/home/madalena/Documents/projeto/train_set/"

for i in os.listdir(input_path):
    os.chdir(output_path)
    print(i)
    os.mkdir(i)
    os.chdir(i)
    image=im.open(input_path+i)
# Augmentation functions can accept image paths as input and
# always return the resulting augmented PIL Image
    aug1 = imaug.RandomNoise(mean=0.0, var=0.01, seed=42, p=1.0)
    image1=aug1(image)
    image1.save(i+"_1.png")
    
    aug2=imaug.RandomRotation(min_degrees=0.0, max_degrees=180.0, p=1.0)
    image2=aug2(image1)
    image2.save(i+"_2.png")
    
    aug3=imaug.OverlayStripes(line_width=0.5, line_color=(255, 255, 255), line_angle=40, line_density=0.3, line_type='solid', line_opacity=0.8, p=1)
    image3=aug3(image1)
    image3.save(i+"_3.png")
    
    aug4=imaug.Contrast(factor=0.7, p=1)
    image4=aug4(image2)
    image4.save(i+"_4.png")
    
    aug5=imaug.PerspectiveTransform(sigma=30.0, dx=0.0, dy=0.0, seed=42, p=1)
    image5=aug5(image4)
    image5.save(i+"_5.png")
    
    aug6=imaug.RandomBlur(min_radius=0.0, max_radius=2.0, p=1)
    image6=aug6(image)
    image6.save(i+"_6.png")
    
    aug7=imaug.RandomPixelization(min_ratio=0.1, max_ratio=1.0, p=0.7)
    image7=aug7(image6)
    image7.save(i+"_7.png")

João Paulo Silva Barraca.png
Diogo Nuno Pereira Gomes.png
Luís Miguel de Noronha Pessoa de Amorim.png
José Manuel Matos Moreira.png
António José Nunes Navarro Rodrigues.png
Ana Maria Perfeito Tomé.png
Paulo Jorge Salvador Serra Ferreira.png
Armando Carlos Domingues da Rocha.png
Tiago Marques Godinho.png
Maria Beatriz Alves de Sousa Santos.png
José Luis Guimarães Oliveira.png
Susana de Jesus Mota.png
Joaquim Manuel Henriques de Sousa Pinto.png
Arnaldo Silva Rodrigues de Oliveira.png
Sérgio Guilherme Aleixo de Matos.png
Artur José Carneiro Pereira.png
Paulo Jorge de Campos Bartolomeu.png
Mário Luís Pinto Antunes.png
Miguel Augusto Mendes Oliveira e Silva.png
Atílio Manuel da Silva Gameiro.png
Armando Humberto Moreira Nolasco Pinto.png
Paulo Miguel de Jesus Dias.png
António Rui de Oliveira e Silva Borges.png
Ayman Radwan.png
Paulo Jorge dos Santos Gonçalves Ferreira.png
Luís António Bastião Silva.png
Victor Manuel L. Macedo Marques.png
Helder Troca Zagalo.png
Osvaldo Manuel da Rocha Pache